# Sheet
Deze sheet geeft een overzicht van uitkomsten van de verschillende p waardes voor één enkele actor

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import ipywidgets as widget
%matplotlib inline

In [2]:
conn = sqlite3.connect('data/database.db')

In [3]:
c = conn.cursor()
cursor = c.execute('SELECT dataset.name, dataset.id FROM dataset ORDER BY dataset.name')
dataset_dropdown = widget.Dropdown(
    options=dict(cursor.fetchall()),
    description='Select dataset:',
    disabled=False    
)

dataset_dropdown

Dropdown(description='Select dataset:', options={'kopenhagen': 1}, value=1)

In [6]:
c = conn.cursor()
cursor = c.execute('SELECT a.name, a.id FROM actor a WHERE a.data_set_id = ? ORDER BY a.name ', (dataset_dropdown.value,))
actor_dropdown = widget.Dropdown(
    options=dict(cursor.fetchall()),
    description='Select dataset:',
    disabled=False    
)

actor_dropdown

Dropdown(description='Select dataset:', options={'allianceofsmallislandstates': 11, 'australia': 3, 'brazil': 8, 'canada': 4, 'chinaindia': 7, 'developingcountries': 10, 'euinclnorway': 2, 'japan': 5, 'leastdevelopedcountries': 9, 'russia': 6, 'usa': 1}, value=11)

In [8]:
df = pd.read_sql("""
SELECT
  ai.position,
  i.name     AS issue,
  i2.pointer AS iteration,
  m.p
FROM actorissue ai
  LEFT JOIN actor a ON ai.actor_id = a.id
  LEFT JOIN issue i ON ai.issue_id = i.id
  LEFT JOIN iteration i2 ON ai.iteration_id = i2.id
  LEFT JOIN repetition r ON i2.repetition_id = r.id
  LEFT JOIN modelrun m ON r.model_run_id = m.id
WHERE ai.type = 'before' AND a.id = ?;
""", 
                 conn, 
                 params=(actor_dropdown.value,),
                 index_col='p',                  
                 columns=['position']
                )

def avg_std(series):
    avg = np.average(series)
    std = np.std(series)
    
    return '{0} ({1})'.format(round(avg, 2), round(std, 5)) 

table = pd.pivot_table(df, index=['issue','p'], columns=['iteration'],aggfunc=[avg_std])
table

avg_std                                                \
                  position                                                 
iteration                0            1               2                3   
issue       p                                                              
Commitments 0   12.5 (0.0)  49.25 (0.0)     28.57 (0.0)  47.21 (0.74166)   
Control     0   50.0 (0.0)  51.13 (0.0)     52.04 (0.0)      52.76 (0.0)   
DevlopC2020 0  66.67 (0.0)  66.46 (0.0)     66.46 (0.0)      66.14 (0.0)   
Domestred   0   62.5 (0.0)   62.5 (0.0)     62.46 (0.0)      62.13 (0.0)   
Extra       0  44.44 (0.0)  63.33 (0.0)      75.8 (0.0)      84.03 (0.0)   
MRVdecrease 0   75.0 (0.0)   52.5 (0.0)     36.75 (0.0)   55.26 (0.2891)   
Newtreaty   0  44.44 (0.0)  63.33 (0.0)  42.36 (0.4448)  61.65 (0.29357)   

                                                                  \
                                                                   
iteration                    4                5                6   
issue       p                                                      
Commitments 0  27.51 (0.43113)  17.11 (4.72628)  31.78 (1.86331)   
Control     0  53.29 (0.06208)  53.29 (0.06296)  53.29 (0.06296)   
DevlopC2020 0      66.14 (0.0)  65.77 (0.00103)   65.71 (0.1908)   
Domestred   0      61.48 (0.0)  60.85 (0.21091)  59.88 (0.55819)   
Extra       0  80.24 (0.56397)  86.96 (0.37222)  91.14 (1.26191)   
MRVdecrease 0  38.76 (0.20565)   54.83 (1.3914)  38.52 (0.96983)   
Newtreaty   0  42.09 (0.86666)  61.35 (0.57533)  43.96 (4.02794)   

                                                                  
                                                                  
iteration                    7                8                9  
issue       p                                                     
Commitments 0  18.76 (1.08097)  13.86 (6.04054)  23.35 (4.63791)  
Control     0  56.05 (2.55602)  59.22 (2.76286)  61.79 (3.11162)  
DevlopC2020 0  65.69 (0.22957)  65.69 (0.22957)  65.69 (0.22957)  
Domestred   0  56.31 (1.00716)  54.99 (1.00257)  54.22 (1.67646)  
Extra       0  87.11 (0.52249)  91.44 (0.35808)  94.04 (0.97702)  
MRVdecrease 0  51.94 (5.11332)  41.99 (6.85649)  47.68 (5.63567)  
Newtreaty   0  58.92 (7.38628)  48.46 (6.49682)  52.8 (10.88498)